In [ ]:
import cv2
import face_recognition
import numpy as np
import os

# 登録された顔データと名前
registered_encodings = []
registered_names = []

# 顔画像が保存されているディレクトリ
faces_dir = "/Users/daigo/Documents/program/test/Advance_Vision/known_faces/"

# ディレクトリ内のすべての画像ファイルを処理
for filename in os.listdir(faces_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # ファイルパスを作成
        image_path = os.path.join(faces_dir, filename)

        # ファイル名から名前を取得
        name = os.path.splitext(filename)[0]
        registered_names.append(name)

        # 顔画像をエンコード
        image = face_recognition.load_image_file(image_path)
        encoding = face_recognition.face_encodings(image)[0]
        registered_encodings.append(encoding)

# カメラを初期化
cap = cv2.VideoCapture(0)

# 解像度を設定
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

if not cap.isOpened():
    print("カメラが見つからないか、開けません。")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("フレームの取得に失敗しました。")
        break

    # フレームをRGBに変換
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 顔の場所を検出
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # 顔を登録データと比較
        matches = face_recognition.compare_faces(registered_encodings, face_encoding)
        name = "Unknown"

        # 最も近い登録顔を探す
        face_distances = face_recognition.face_distance(registered_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = registered_names[best_match_index]

        # 矩形と名前を描画
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # フレームを表示
    cv2.imshow("Camera", frame)

    # 'q'を押すと終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# カメラとウィンドウを解放
cap.release()
cv2.destroyAllWindows()


2024-12-15 00:05:42.252 python[44974:456908] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
